In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = saw
band = [6]
commitnumber = de5d43e
molecule = {'co2': 0.0004}
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band4
  atmpro = mls
  band = 4
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.5e-24
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [1, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 980
  vmin = 800
  w_diffuse = [(1.75,), (1.66, 1.9)]
  wgt = [(0.75,), (0.75, 0.95)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0 -0.000151                  NaN       NaN
0.000312      NaN       NaN                  1.0 -0.000261
0.000750      2.0 -0.000123                  2.0 -0.000351
0.001052      3.0 -0.000050                  3.0 -0.000152
0.001476      4.0  0.000048                  4.0 -0.000082
0.002070      5.0  0.000177                  5.0  0.000003
0.002904      6.0  0.000344                  6.0  0.000107
0.004074      7.0  0.000555                  7.0  0.000242
0.005714      8.0  0.000819                  8.0  0.000422
0.008015      9.0  0.001146                  9.0  0.000648
0.011243     10.0  0.001548                 10.0  0.000924
0.015771     11.0  0.002039                 11.0  0.001258
0.022122     12.0  0.002634                 12.0  0.001660
0.031031     13.0  0.003346                 13.0  0.002143
0.043528     14.0  0.004091                 14.0  0.002646
0.061057     15.0  0.004751                 15.0  0.003093
0.085645     16.0  0.005376                 16.0  0.003524
0.120136     17.0  0.006058                 17.0  0.003991
0.168516     18.0  0.006800                 18.0  0.004485
0.236378     19.0  0.007610                 19.0  0.005002
0.331549     20.0  0.008488                 20.0  0.005545
0.465100     21.0  0.009333                 21.0  0.006116
0.652400     22.0  0.008561                 22.0  0.005744
0.915100     23.0  0.006058                 23.0  0.004138
1.283650     24.0  0.003809                 24.0  0.002626
1.800600     25.0  0.002257                 25.0  0.001557
2.525700     26.0  0.001217                 26.0  0.000830
3.542800     27.0  0.000544                 27.0  0.000350
4.969550     28.0  0.000169                 28.0  0.000077
6.970850     29.0 -0.000013                 29.0 -0.000061
9.778100     30.0 -0.000119                 30.0 -0.000144
13.715850    31.0 -0.000192                 31.0 -0.000204
19.239350    32.0 -0.000242                 32.0 -0.000245
26.987250    33.0 -0.000261                 33.0 -0.000261
37.855300    34.0 -0.000258                 34.0 -0.000255
53.100050    35.0 -0.000250                 35.0 -0.000259
73.887500    36.0 -0.000245                 36.0 -0.000264
97.662500    37.0 -0.000246                 37.0 -0.000261
121.437500   38.0 -0.000258                 38.0 -0.000263
145.212500   39.0 -0.000277                 39.0 -0.000264
168.987500   40.0 -0.000294                 40.0 -0.000265
192.762500   41.0 -0.000309                 41.0 -0.000268
216.537500   42.0 -0.000322                 42.0 -0.000277
240.312500   43.0 -0.000334                 43.0 -0.000287
264.087500   44.0 -0.000345                 44.0 -0.000296
287.862500   45.0 -0.000352                 45.0 -0.000299
311.637500   46.0 -0.000330                 46.0 -0.000277
335.412500   47.0 -0.000257                 47.0 -0.000201
359.187500   48.0 -0.000134                 48.0 -0.000083
382.962500   49.0  0.000024                 49.0  0.000067
406.737500   50.0  0.000218                 50.0  0.000250
430.512500   51.0  0.000447                 51.0  0.000462
454.287500   52.0  0.000713                 52.0  0.000709
478.062500   53.0  0.001013                 53.0  0.000994
501.837500   54.0  0.001347                 54.0  0.001316
525.612500   55.0  0.001713                 55.0  0.001645
549.387500   56.0  0.002105                 56.0  0.002005
573.162500   57.0  0.002518                 57.0  0.002391
596.937500   58.0  0.002945                 58.0  0.002799
620.712500   59.0  0.003378                 59.0  0.003222
644.487500   60.0  0.003809                 60.0  0.003631
668.262500   61.0  0.004232                 61.0  0.004021
692.037500   62.0  0.004642                 62.0  0.004408
715.812500   63.0  0.005037                 63.0  0.004787
739.587500   64.0  0.005418                 64.0  0.005159
76

# Fluxes by Layer

CRD                              CLIRAD  \
                         flug          fldg      fnetg       flug   
pressure     level                                                  
0.000000e+00 1     -32.785542  0.000000e+00 -32.785542        NaN   
1.000000e-08 1            NaN           NaN        NaN -32.793273   
6.244000e-04 2     -32.785542  3.465142e-08 -32.785542 -32.793273   
8.759000e-04 3     -32.785542  4.994792e-08 -32.785542 -32.793273   
1.228600e-03 4     -32.785542  7.572967e-08 -32.785542 -32.793273   
1.723400e-03 5     -32.785542  1.192048e-07 -32.785542 -32.793273   
2.417400e-03 6     -32.785542  1.923294e-07 -32.785542 -32.793273   
3.390900e-03 7     -32.785542  3.150608e-07 -32.785541 -32.793274   
4.756500e-03 8     -32.785542  5.205689e-07 -32.785541 -32.793274   
6.672000e-03 9     -32.785542  8.638079e-07 -32.785541 -32.793274   
9.358900e-03 10    -32.785542  1.435846e-06 -32.785541 -32.793274   
1.312780e-02 11    -32.785543  2.387148e-06 -32.785540 -32.793274   
1.841450e-02 12    -32.785543  3.965853e-06 -32.785539 -32.793275   
2.583020e-02 13    -32.785543  6.580245e-06 -32.785537 -32.793275   
3.623230e-02 14    -32.785543  1.089771e-05 -32.785532 -32.793276   
5.082340e-02 15    -32.785543  1.788203e-05 -32.785525 -32.793276   
7.129060e-02 16    -32.785543  2.880608e-05 -32.785514 -32.793277   
1.000000e-01 17    -32.785541  4.559269e-05 -32.785496 -32.793277   
1.402710e-01 18    -32.785538  7.133229e-05 -32.785467 -32.793277   
1.967600e-01 19    -32.785532  1.107098e-04 -32.785421 -32.793275   
2.759970e-01 20    -32.785521  1.708278e-04 -32.785350 -32.793271   
3.871000e-01 21    -32.785500  2.623075e-04 -32.785238 -32.793264   
5.431000e-01 22    -32.785466  3.999940e-04 -32.785066 -32.793249   
7.617000e-01 23    -32.785422  5.783980e-04 -32.784844 -32.793233   
1.068500e+00 24    -32.785391  7.671885e-04 -32.784624 -32.793228   
1.498800e+00 25    -32.785382  9.524658e-04 -32.784430 -32.793244   
2.102400e+00 26    -32.785400  1.132183e-03 -32.784268 -32.793283   
2.949000e+00 27    -32.785451  1.304826e-03 -32.784146 -32.793352   
4.136600e+00 28    -32.785539  1.469815e-03 -32.784070 -32.793453   
5.802500e+00 29    -32.785672  1.635561e-03 -32.784036 -32.793595   
8.139200e+00 30    -32.785860  1.819916e-03 -32.784040 -32.793790   
1.141700e+01 31    -32.786123  2.037575e-03 -32.784086 -32.794055   
1.601470e+01 32    -32.786491  2.300315e-03 -32.784191 -32.794416   
2.246400e+01 33    -32.787003  2.627677e-03 -32.784376 -32.794908   
3.151050e+01 34    -32.787737  3.081458e-03 -32.784656 -32.795595   
4.420010e+01 35    -32.788824  3.780987e-03 -32.785043 -32.796571   
6.200000e+01 36    -32.790446  4.875453e-03 -32.785571 -32.798062   
8.577500e+01 37    -32.792752  6.490182e-03 -32.786262 -32.800253   
1.095500e+02 38    -32.795177  8.223432e-03 -32.786953 -32.802566   
1.333250e+02 39    -32.797681  9.999898e-03 -32.787681 -32.804929   
1.571000e+02 40    -32.800238  1.177659e-02 -32.788462 -32.807274   
1.808750e+02 41    -32.802841  1.355180e-02 -32.789289 -32.809603   
2.046500e+02 42    -32.805484  1.532556e-02 -32.790158 -32.811932   
2.284250e+02 43    -32.808163  1.709794e-02 -32.791065 -32.814319   
2.522000e+02 44    -32.810875  1.886901e-02 -32.792006 -32.816780   
2.759750e+02 45    -32.813617  2.063949e-02 -32.792977 -32.819299   
2.997500e+02 46    -32.816396  2.242867e-02 -32.793968 -32.821867   
3.235250e+02 47    -32.819273  2.437468e-02 -32.794898 -32.824531   
3.473000e+02 48    -32.822320  2.669928e-02 -32.795621 -32.827308   
3.710750e+02 49    -32.825543  2.954441e-02 -32.795998 -32.830222   
3.948500e+02 50    -32.828906  3.297691e-02 -32.795930 -32.833266   
4.186250e+02 51    -32.832372  3.705609e-02 -32.795316 -32.836390   
4.424000e+02 52    -32.835896  4.184032e-02 -32.794056 -32.839529   
4.661750e+02 53    -32.839435  4.738594e-02 -32.792049 -32.842669   
4.899500e+02 54    -32.842942  5.374591e-02 -32.789196 -32.845784   
5.137250e+02 55    -32.8

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')